In [3]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async, RateLimiter, send_message_open_router
from pprint import pprint
from algo import base_start_dialog_algorithm, base_continue_dialog_algorithm, simulate_user_response_prompt, \
    student_card_template, send_message, extract_student_health_data_prompt, \
    generate_readable_history_from_end,  update_message_history_with_system_message, convert_gemini_history_to_open_router


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Start algo question answer scenario

In [35]:
class ConversationManager:
    def __init__(self, verbose=False):
        self.rate_limiter = RateLimiter(13, 14)
        self.verbose = verbose

        # user info
        self.telegram_user_id = []

        # conversaion info
        self.message_history = []
        self.usage_history = []
        self.dialog_count = 0
        self.dialog_status = 'continue'
        self.extraction_usage_history = []

        # extraction
        self.student_card = {}


    async def start_conversation(self):
        """
        Начинает диалог с заготовленных сообщений
        """
        self.student_card = student_card_template().copy()
        prompted_message = base_start_dialog_algorithm().replace("$STUDENT_CARD", str(self.student_card))
        self.message_history.append({"role": "user", "parts": [prompted_message]})
        self.message_history.append({"role": "model", "parts": ['Здравствуйте! Меня зовут Аика, и я очень рада приветствовать вас в нашем диалоге по улучшению здоровья. Прежде чем начать, мне необходимо проверить ваш промокод, чтобы получить доступ к созданию алгоритма. Ваш промокод находится в 9 уроке.']})


    async def process_model_interaction(self, user_query):
        if user_query.strip() == '':
            user_query = "Расскажи шутку. А потом давай продолжим"
        
        await self.get_message_from_model(user_query, provider='gemini')
        model_message = self.message_history[-1]['parts'][0]
        return model_message

    
    async def get_message_from_model(self, user_query, provider='gemini'):
        self.message_history.append({"role": "user", "parts": [user_query]})

        if provider == 'gemini':
            response = await send_message_to_gemini_async(self.message_history, rate_limiter=self.rate_limiter, generation_params={"temperature": 0.2, "top_k":3})
        elif provider == 'router_sonnet':
            openrouter_history = convert_gemini_history_to_open_router(self.message_history)
            response = await send_message_open_router(openrouter_history, rate_limiter=self.rate_limiter, model="anthropic/claude-3-sonnet")
        

        input_tokens = response.get('input_tokens')
        output_tokens = response.get('output_tokens')
        answer_text = response.get('text_response')

        self.message_history.append({"role": "model", "parts": [answer_text]})
        self.usage_history.append({"provider": provider, "query": user_query, "answer": answer_text, "input_tokens": input_tokens, "output_tokens": output_tokens})
        if self.verbose:
            print('\n\nUSER:\n', user_query)
            print('\n\nMODEL:\n', answer_text)
            print({"provider": provider, "input_tokens": input_tokens, "output_tokens": output_tokens})


    async def update_student_card_from_message_history(self, dialogs_num=5):
        model_message = self.message_history[-1]['parts'][0]

        if (self.dialog_count % 4 == 0 and self.dialog_count > 0) or ('next_step' in model_message.lower()):
        
            readable_history = await generate_readable_history_from_end(messages=self.message_history, dialogs_num=dialogs_num)
            extract_student_health_data_prompted = extract_student_health_data_prompt().replace("$DIALOG", readable_history).replace("$STUDENT_CARD", str(self.student_card))
            messages = [{"role": "model", "parts": [extract_student_health_data_prompted]}]
            response = await send_message_to_gemini_async(messages, rate_limiter=self.rate_limiter, generation_params={"temperature": 0, "top_k":1})

            input_tokens = response.get('input_tokens')
            output_tokens = response.get('output_tokens')
            answer_text = response.get('text_response')

            try:
                # UPDATE STUDENT CARD
                new_info = json.loads(answer_text.strip('```json').strip('```'))
                self.student_card.update(new_info)
            except:
                new_info = f'Не найдена. Текст ответа: {response}'

            if self.verbose:
                print('response:', response)

            # ADD TO HISTORY
            self.extraction_usage_history.append({"provider": 'gemini', "query": extract_student_health_data_prompted, "answer": answer_text, "input_tokens": input_tokens, "output_tokens": output_tokens})

    
    async def update_dialog_status(self):
        model_message = self.message_history[-1]['parts'][0]
        
        if 'terminate' in model_message.lower():   
            self.dialog_status = 'terminate'
        elif 'next_step' in model_message.lower(): 
            self.dialog_status = 'next_step'


    async def simulate_user_response(self):
        user_query = simulate_user_response_prompt().replace("$DOCTOR_QUESTION", self.message_history[-1]['parts'][0])
        response_user = await send_message_to_gemini_async(history = [{'role': 'user', 'parts': [user_query]}])
        user_query = response_user.get('text_response')
        return user_query
    
    async def send_message_to_user_telegram(self, model_message):
        # self.user_id
        pass

    async def run(self):
        await self.start_conversation()
        while self.dialog_status not in ['terminate', 'next_step']:
            if self.verbose: 
                print(f'dialog_count: {self.dialog_count}')
            user_query = await self.simulate_user_response()
            model_message = await self.process_model_interaction(user_query)
            await self.send_message_to_user_telegram(model_message)
            await self.update_dialog_status()
            await self.update_student_card_from_message_history()

            self.dialog_count += 1

# conv_manager = ConversationManager()
# try:
#     await conv_manager.run()
# except KeyboardInterrupt:
#     print('Stopping due to KeyboardInterrupt')

conv_manager = ConversationManager(verbose=True)


await conv_manager.start_conversation()
while conv_manager.dialog_status not in ['terminate', 'next_step']:
    if conv_manager.verbose: 
        print(f'dialog_count: {conv_manager.dialog_count}')
    user_query = await conv_manager.simulate_user_response()
    model_message = await conv_manager.process_model_interaction(user_query)
    await conv_manager.send_message_to_user_telegram(model_message)
    await conv_manager.update_dialog_status()
    await conv_manager.update_student_card_from_message_history()




dialog_count: 0


USER:
 Q6sg5


MODEL:
 Спасибо за предоставленный промокод. Теперь я могу приступить к сбору необходимой информации для создания вашего алгоритма. Давайте начнем с вашего имени и возраста.
{'provider': 'gemini', 'input_tokens': 4, 'output_tokens': 34}


TypeError: ConversationManager.send_message_to_user_telegram() missing 1 required positional argument: 'model_message'

In [31]:
student_card = conv_manager.student_card
message_history = conv_manager.message_history
usage_history = conv_manager.usage_history
extraction_usage_history = conv_manager.extraction_usage_history

student_card

{'description of card': {'value': 'карточка о здоровье, хранических заболеваниях, выбранном лечении'},
 'Промокод': {'value': ''},
 'Имя и возраст': {'value': 'Анна, 27 лет'},
 'Цели по здоровью': {'value': 'Лучше понимать свое тело и потребности в питании'},
 'Артериальное давление': {'value': '120/80, гипертония под контролем'},
 'Тип сокращения желчного пузыря': {'value': 'Нормокинетический'},
 'Выбранная трава желчегонная': {'value': 'Тысячелистник'},
 'Желчегонная гимнастика': {'value': 'Будет выполнять в ближайшее время'},
 'Витамин Д': {'value': '400 МЕ в день'},
 'Время принятия лимфатического душа': {'value': 'Вечером'},
 'Результаты копрограммы': {'value': '',
  'instruction': 'Спросить о результатах копрограммы, если они имеются.'},
 'Хронические заболевания или противопоказания': {'value': 'легкая астма'}}

In [46]:
# TOODO если желчный удален то не спрашивать про желчегонную траву и желчегонную гимнастику


# USAGES

In [71]:
# SONNET
import pandas as pd
us = pd.DataFrame(usage_history)
print("us['input_tokens'].sum()", us['input_tokens'].sum())
print("us['output_tokens'].sum()", us['output_tokens'].sum())
print("price", us['input_tokens'].sum()/1000 * 0.003 + us['output_tokens'].sum()/1000 * 0.015)
us

us['input_tokens'].sum() 47084
us['output_tokens'].sum() 2987
price 0.18605700000000003


,input_tokens,output_tokens,conversation_tokens
0,1427,127,1554
1,1561,84,3199
2,1693,130,5022
3,1860,170,7052
4,2068,97,9217
5,2238,241,11696
6,2522,202,14420
7,2780,200,17400
8,3007,212,20619
9,3237,233,24089


In [54]:
# GEMINI
us.to_csv('gemini_token_usage_16msgs.csv')
us

,input_tokens,output_tokens,conversation_tokens
0,979,47,1026
1,18,30,1074
2,4,51,1129
3,11,39,1179
4,14,87,1280
5,3,60,1343
6,5,86,1434
7,6,54,1494
8,4,141,1639
9,17,108,1764


{}